In [5]:
from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.solvers.mip_solver import mip_solve
from cloudmanufacturing.validation import construct_delta, objvalue

from joblib import Parallel, delayed
from tqdm import trange
import pandas as pd
import numpy as np

c:\Users\z004w9ze\AppData\Local\miniconda3\envs\graph\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Fatahi

In [5]:
dataset = read_fatahi_dataset("../data/fatahi.xlsx")

100%|██████████| 18/18 [00:00<?, ?it/s]


In [6]:
def fatahi_problem(i, problem):
    print(problem['n_operations'])
    print(problem['n_suboperations'])
    print(problem['n_cities'])
    delta, gamma, status, value = mip_solve(problem)

    if status.name == 'OPTIMAL':
        np.save(f'./fatahi_solutions/delta_{i}_op.npy', delta)
        np.save(f'./fatahi_solutions/gamma_{i}_op.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    if status.name == 'FEASIBLE':
        np.save(f'./fatahi_solutions/delta_{i}_fb.npy', delta)
        np.save(f'./fatahi_solutions/gamma_{i}_fb.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    print(i," wasn't solved")
    return None

In [7]:
sum_info = Parallel(n_jobs=-1)(
    delayed(fatahi_problem)(i, dataset[i]) for i in trange(len(dataset))
)
sum_info = [info for info in sum_info if info is not None]

pd.DataFrame(sum_info).to_csv('fatahi_info.csv', index=False)

100%|██████████| 18/18 [00:00<00:00, 19533.63it/s]


In [8]:
pd.DataFrame(sum_info)

,i,value,objvalue,status
0,0,4.405050e+05,4.405050e+05,FEASIBLE
1,1,1.439140e+05,1.439140e+05,FEASIBLE
2,2,1.736413e+05,1.736413e+05,FEASIBLE
3,3,1.260019e+05,1.260019e+05,FEASIBLE
4,4,1.287558e+06,1.287558e+06,FEASIBLE
5,5,7.724917e+05,7.724917e+05,FEASIBLE
6,7,6.290782e+05,6.290782e+05,FEASIBLE
7,10,9.242748e+05,9.242748e+05,FEASIBLE
8,11,1.973185e+06,1.973185e+06,FEASIBLE
9,12,3.322294e+06,3.322294e+06,FEASIBLE


### Train dataset

In [6]:
dataset_train = read_fatahi_dataset("../data/train_data.xlsx")

100%|██████████| 100/100 [00:36<00:00,  2.74it/s]


In [9]:
def train_problem(i, problem):
    delta, gamma, status, value = mip_solve(problem)

    if status.name == 'OPTIMAL':
        np.save(f'./train_solutions/delta_{i}_op.npy', delta)
        np.save(f'./train_solutions/gamma_{i}_op.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    if status.name == 'FEASIBLE':
        np.save(f'./train_solutions/delta_{i}_fb.npy', delta)
        np.save(f'./train_solutions/gamma_{i}_fb.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    print(i," wasn't solved")
    return None

In [10]:
sum_info_train = Parallel(n_jobs=-1)(
    delayed(train_problem)(i, dataset_train[i]) for i in trange(len(dataset_train))
)
sum_info_train = [info for info in sum_info_train if info is not None]

pd.DataFrame(sum_info_train).to_csv('train_info.csv', index=False)

KeyboardInterrupt: 

#### Data generation

In [1]:
import random
import openpyxl
from cloudmanufacturing.data_generation import create_excel_table
random.seed(42)

In [2]:
N = 100
stop_count = 100

In [3]:
file_path = '../data/train_data.xlsx'

trios = []
for i in range(N):
    trios.append(tuple((random.randint(5,10),random.randint(10,15),random.randint(10,15))))
trios.sort()

wb = openpyxl.Workbook()
del wb['Sheet']

for n_operations, n_suboperations, n_cities in trios:
    wb = create_excel_table(wb, n_operations, n_suboperations, n_cities, n_problem=1)

wb.save(file_path)

### junk

In [ ]:
# sum_info = []
# for i in trange(len(dataset)):
#     problem = dataset[i]
#     delta, gamma, status, value = mip_solve(problem)
#     if status.name == 'OPTIMAL':
#         np.save(f'./mip_solutions/delta_{i}_opt.npy', delta)
#         np.save(f'./mip_solutions/gamma_{i}_opt.npy', gamma)
#         sum_info.append({'value':value,
#                         'objvalue':objvalue(problem, gamma, delta),
#                         'status':status
#                         })
#     if status.name == 'FEASIBLE':
#         np.save(f'./mip_solutions/delta_{i}_opt.npy', delta)
#         np.save(f'./mip_solutions/gamma_{i}_opt.npy', gamma)
#         sum_info.append({'value':value,
#                         'objvalue':objvalue(problem, gamma, delta),
#                         'status':status
#                         })
# pd.DataFrame(sum_info).to_csv('./mip_solutions/info.csv',index=False)